In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/air-quality-and-health-impact-dataset/air_quality_health_impact_data.csv


In [2]:
df = pd.read_csv('/kaggle/input/air-quality-and-health-impact-dataset/air_quality_health_impact_data.csv')
pd.set_option('max_colwidth', 100)
df.head(5)

,RecordID,AQI,PM10,PM2_5,NO2,SO2,O3,Temperature,Humidity,WindSpeed,RespiratoryCases,CardiovascularCases,HospitalAdmissions,HealthImpactScore,HealthImpactClass
0,1,187.270059,295.853039,13.038560,6.639263,66.161150,54.624280,5.150335,84.424344,6.137755,7,5,1,97.244041,0.0
1,2,475.357153,246.254703,9.984497,16.318326,90.499523,169.621728,1.543378,46.851415,4.521422,10,2,0,100.000000,0.0
2,3,365.996971,84.443191,23.111340,96.317811,17.875850,9.006794,1.169483,17.806977,11.157384,13,3,0,100.000000,0.0
3,4,299.329242,21.020609,14.273403,81.234403,48.323616,93.161033,21.925276,99.473373,15.302500,8,8,1,100.000000,0.0
4,5,78.009320,16.987667,152.111623,121.235461,90.866167,241.795138,9.217517,24.906837,14.534733,9,0,1,95.182643,0.0


### Preprocessing(Ön İşleme) Aşamaları

1) Eksik Verileri(NA) Kontrol Etme ve Doldurma:

Veri setinde eksik veri bulunmuyor, bu yüzden bu adımı atlıyorum

2) Özellik Ölçekleme (Feature Scaling):

Özelliklerin farklı ölçeklerde olması, özellikle mesafe tabanlı algoritmalarda (KNN, SVM) model performansını etkileyebilir. Bu nedenle, verileri ölçeklendirmek faydalı olabilir.

**Not:** StandardScaler veya MinMaxScaler kullanarak verileri ölçeklendireceğim.

3) Kategorik Verilerin Dönüştürülmesi:

Veri setinde sadece sayısal veriler var, bu nedenle bu adımı atlıyorum.

4) Hedef Değişken (Label) Dönüştürme:

HealthImpactClass hedef değişkenimiz olacak. Bu değişkenin sınıflandırma için uygun olup olmadığını kontrol edelim.

In [3]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)


check_df(df)

##################### Shape #####################
(5811, 15)
##################### Types #####################
RecordID                 int64
AQI                    float64
PM10                   float64
PM2_5                  float64
NO2                    float64
SO2                    float64
O3                     float64
Temperature            float64
Humidity               float64
WindSpeed              float64
RespiratoryCases         int64
CardiovascularCases      int64
HospitalAdmissions       int64
HealthImpactScore      float64
HealthImpactClass      float64
dtype: object
##################### Head #####################
   RecordID         AQI        PM10       PM2_5         NO2        SO2  \
0         1  187.270059  295.853039   13.038560    6.639263  66.161150   
1         2  475.357153  246.254703    9.984497   16.318326  90.499523   
2         3  365.996971   84.443191   23.111340   96.317811  17.875850   
3         4  299.329242   21.020609   14.273403   81.234403  48.3

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [5]:
# Özellikler (X) ve hedef değişken (y)

X = df.drop('HealthImpactClass', axis=1)
y = df['HealthImpactClass']

In [7]:
# Train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=58)

In [8]:
# Gaussian Naive Bayes modelin oluşturulması ve eğitim

gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [9]:
# Tahmin yapılması

y_pred = gnb.predict(X_test)
print(y_pred)

[0. 0. 0. ... 0. 0. 0.]


In [10]:
# Model performansı değerlendirilmesi

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.8907996560619088
Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.93      0.95       966
         1.0       0.56      0.94      0.70       111
         2.0       0.75      0.54      0.63        56
         3.0       0.00      0.00      0.00        14
         4.0       0.33      0.06      0.11        16

    accuracy                           0.89      1163
   macro avg       0.52      0.49      0.48      1163
weighted avg       0.89      0.89      0.89      1163

Confusion Matrix:
[[901  62   3   0   0]
 [  7 104   0   0   0]
 [  8  16  30   0   2]
 [  6   1   7   0   0]
 [ 13   2   0   0   1]]


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
